In [275]:
import requests
import pandas as pd
import re
import numpy as np
from fuzzywuzzy import fuzz
from thefuzz import fuzz, process
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
import numpy as np

resp = requests.get('https://www.sec.gov/files/company_tickers.json')
#df= pd.read_json(resp.text)
df = pd.read_json(resp.text, orient ='index')

print(df)

       cik_str ticker                                  title
0       320193   AAPL                             Apple Inc.
1       789019   MSFT                         MICROSOFT CORP
2      1652044  GOOGL                          Alphabet Inc.
3      1018724   AMZN                         AMAZON COM INC
4      1045810   NVDA                            NVIDIA CORP
...        ...    ...                                    ...
11154  1901799  GSRMW       GSR II Meteora Acquisition Corp.
11155  1742927  RVPHW  REVIVA PHARMACEUTICALS HOLDINGS, INC.
11156  1853774  MCAFR    Mountain Crest Acquisition Corp. IV
11157  1853774  MCAFU    Mountain Crest Acquisition Corp. IV
11158  1802457  ORGNW                 Origin Materials, Inc.

[11159 rows x 3 columns]


In [276]:
!pip install fuzzywuzzy
!pip install thefuzz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [277]:
# preprocessing data
df0 = df.copy()
df0['title_0'] = df0['title'].apply(str.lower)
df0['title_0'] = df0['title_0'].apply(lambda x: re.sub(r'[^a-zA-Z0-9]+', ' ', x))
df0['title_0'] = df0['title_0'].apply(lambda x: re.sub("\s\s+", " ", x))
df0['title_0'] = df0['title_0'].apply(lambda x: x.strip())
df0['ticker_0'] = df0['ticker'].apply(str.lower)
df0['ticker_0'] = df0['ticker_0'].apply(lambda x: re.sub(r'[^a-zA-Z0-9]+', ' ', x))
df0['ticker_0'] = df0['ticker_0'].apply(lambda x: re.sub("\s\s+", " ", x))
df0['ticker_0'] = df0['ticker_0'].apply(lambda x: x.strip())
df0

,cik_str,ticker,title,title_0,ticker_0
0,320193,AAPL,Apple Inc.,apple inc,aapl
1,789019,MSFT,MICROSOFT CORP,microsoft corp,msft
2,1652044,GOOGL,Alphabet Inc.,alphabet inc,googl
3,1018724,AMZN,AMAZON COM INC,amazon com inc,amzn
4,1045810,NVDA,NVIDIA CORP,nvidia corp,nvda
...,...,...,...,...,...
11154,1901799,GSRMW,GSR II Meteora Acquisition Corp.,gsr ii meteora acquisition corp,gsrmw
11155,1742927,RVPHW,"REVIVA PHARMACEUTICALS HOLDINGS, INC.",reviva pharmaceuticals holdings inc,rvphw
11156,1853774,MCAFR,Mountain Crest Acquisition Corp. IV,mountain crest acquisition corp iv,mcafr
11157,1853774,MCAFU,Mountain Crest Acquisition Corp. IV,mountain crest acquisition corp iv,mcafu


In [278]:
# partial_ratio can be easily inluenced by small character word and thus have 100 similarity score. Replace ticker with less than 3 character with a random word with lenth 4.
df0['ticker_00'] = df0['ticker_0'].apply(lambda x: x if len(x) >3 else "xyzp")

In [279]:
df0['Suffix'] = df0['title_0'].str.split().str[-1]

In [280]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [281]:
companies = set(df0['title_0'].values.tolist())
companies = list(companies)
company_tokens = [set(word for word in word_tokenize(company)) for company in companies]

In [282]:
componyticker = set(df0["ticker_0"].values.tolist())
componyticker = list(componyticker1)
#companyticker_tokens = set(componyticker)

**Fuzzy Matching:**

The fuzzywuzzy library provides several methods that use the Levenshtein distance and Jaro-Winkler distance algorithms to compute the similarity between two strings.

fuzz.ratio method uses the Levenshtein distance algorithm to calculate the difference between two strings. It computes the ratio of the number of matching characters to the total number of characters in the two strings, expressed as a percentage.

fuzz.partial_ratio method uses partial string matching to compute the similarity ratio. It does not use the Levenshtein distance algorithm.

fuzz.token_sort_ratio and fuzz.token_set_ratio methods use the Jaro-Winkler distance algorithm to compare tokenized versions of the input strings. fuzz.token_sort_ratio sorts the tokens in each string alphabetically before comparing them, while fuzz.token_set_ratio compares the sets of unique tokens in the two strings.

The Jaro-Winkler distance algorithm is a variation of the Jaro distance algorithm, which is used to compare two strings character by character, taking into account both the order and the number of characters that match. The Jaro-Winkler distance algorithm is a modified version of the Jaro distance algorithm that gives more weight to matching characters at the beginning of the strings.

partial_ratio works better for case of partial name of the company;

token_sort_ratio works better for case of differnt order of tokens for company names

token_set_ratio better for case of duplicates tokens 

**Tests**

In [393]:
# example
s1 = "Barkshire"
s2 = "Berkshire Hathaway"
print(fuzz.ratio(s1,s2))
print(fuzz.partial_ratio(s1,s2))
print(fuzz.token_sort_ratio(s1,s2))

59
89
59


In [394]:
# example
s1 = "Barkshire"
s2 = "Berkshire"
print(fuzz.ratio(s1,s2))
print(fuzz.partial_ratio(s1,s2))
print(fuzz.token_sort_ratio(s1,s2))

89
89
89


In [395]:
# example
s1 = "Barkshire Hathaway"
s2 = "Hathaway something Berkshire"
print(fuzz.ratio(s1,s2))
print(fuzz.partial_ratio(s1,s2))
print(fuzz.token_sort_ratio(s1,s2))

35
44
74


In [396]:
# example
s1 = "Berkshire"
s2 = "Hathaway Berkshire"
print(fuzz.ratio(s1,s2))
print(fuzz.partial_ratio(s1,s2))
print(fuzz.token_sort_ratio(s1,s2))

67
100
67


In [400]:
# example
s1 = "Berkshire Hathaway"
s2 = "Berkshire something subsidiary ooo"
print(fuzz.ratio(s1,s2))
print(fuzz.partial_ratio(s1,s2))
print(fuzz.token_sort_ratio(s1,s2))

54
67
54


In a word, for fuzzy match, fuzz.token_sort_ratio works better for reverse order of partial word search like "hat berksh" than partial_ratio. Thus decided to use maximum score of token_sort_ratio and partial_ratio.

In [283]:
# preprocessing input
def process_query(query):
  query = query.lower()
  query = query.replace('&', ' and ')
  query = re.sub(r'[^a-zA-Z0-9]+', ' ', query)
  query = re.sub("\s\s+", ' ', query).strip()

  # remove any stopwors like the, a, an
  # Load the stopwords
  stop_words = set(stopwords.words('english'))
  words = query.split()
  filtered_words = [word for word in words if word.lower() not in stop_words]
  # Convert the filtered words back to a string
  filtered_text = ' '.join(filtered_words)
  return filtered_text

In [284]:
def search_fuzz_ticker(query, num_results, threshold): 
  query = process_query(query)
  l1 = process.extract(query,componyticker,limit = num_results, scorer = fuzz.ratio)
  l2 = process.extract(query,componyticker,limit = num_results, scorer = fuzz.token_sort_ratio)
  l3 =  process.extract(query,companies,limit = num_results, scorer = fuzz.partial_ratio)
  l4 = process.extract(query,companies,limit = num_results, scorer = fuzz.token_sort_ratio)
  ls0 = l1+l2+l3+l4
  ls0.sort(key = lambda x: x[1], reverse = True) 

  name = ls0[:num_results]
  ls = []
  for i in range(0,len(name)):
    if name[i][1] >threshold:
      ls.append(name[i][0])
    # creating filters of bool series from isin()
  filter1 = df0["ticker_0"].isin(ls)
  filter2 = df0["title_0"].isin(ls)
  
  # displaying data with both filter applied and mandatory 
  df =df0[filter1 | filter2]
  #return
  return set(df['title'])

## **Conclusion**

This search engine is going to take an input and return X number of similar or matching entities depending on threshold and require number of output.

If the user are certain of input, he or she can set the threshold to be high and required number of output being low and thus get an exact match. 

If the user are not certain of input, he or she can set the threshold to be low and required number of output being high and thus get an exact match. 

The whole search engine is designed based on Levenshtein distance, Jaro-Winkler distance algorithms and partial string matching on ticker and title of companies.


## **Scenarios that this search system should be able to handle:**


**1. Typo**

In [285]:
search_fuzz_ticker('berkshire..  .. ', 5, 80)

{'BERKSHIRE HATHAWAY INC',
 'BERKSHIRE HILLS BANCORP INC',
 'Berkshire Grey, Inc.'}

In [286]:
search_fuzz_ticker('barkshire', 5, 80)

{'BERKSHIRE HATHAWAY INC',
 'BERKSHIRE HILLS BANCORP INC',
 'Berkshire Grey, Inc.'}

**2. Partial Search**

In [287]:
search_fuzz_ticker('berk', 5, 80)

{'BERKLEY W R CORP',
 'BERKSHIRE HATHAWAY INC',
 'BERKSHIRE HILLS BANCORP INC',
 'Berkeley Group Holdings plc',
 'Berkshire Grey, Inc.'}

In [288]:
search_fuzz_ticker('berkshire something', 5, 50)

{'BERKSHIRE HATHAWAY INC',
 'BERKSHIRE HILLS BANCORP INC',
 'Berkshire Grey, Inc.'}

**3. Ticker or abbreviation**

In [289]:
search_fuzz_ticker('google', 5, 80)

{'Alphabet Inc.'}

In [290]:
search_fuzz_ticker('ibm', 5, 80)

{'INTERNATIONAL BUSINESS MACHINES CORP'}

**4. Input with subsidiary names**

In [291]:
search_fuzz_ticker('Berkshire something subsidiary ooo', 5, 50)

{'BERKSHIRE HATHAWAY INC',
 'BOEING CO',
 'Berkshire Grey, Inc.',
 'HERSHEY CO',
 'HireQuest, Inc.'}

# **Testing & Evaluation**

1. **Recall**

Without knowing context of the search, we are hard to know what users are actually search for, thus making it hard to label which output is true positive and false negative. With default of 5 output and 80 similarity score threshold, we can see from the examples above the output is quite relevant with the input. 

2. **Precision**(pair *quality*)

Instead, precision is a better metric than recall, becase we can manually label out of predicted matching entity, what percentage is possiblely related to the input. 

2. **Confidence score** 

Confidence score is calculated by the mixture of partial string matching and Jaro-Winkler distance. Not using Levenshtein distance because through tests, it has low similarity score for partial search. 

# **Additional Thoughts:**

1. In addition to the metrics mentioned above, we can also consider incorporating user feedback to improve the relevance of the returned entities. You can collect user feedback using click-through rates, bounce rates, or other metrics and use this feedback to fine-tune your search engine.

2. Due to nature of the dataset from one data source, most company names are unique, not much of company have shared words and unkown ground truth name. When there are large datasets from multiple sources, we should consider using blocking to cluster similar company names together for entity matching. By doing so, the complexity of pariwide comparisons is reduced from O(n^2), n being the number of records, to O(K*B^2), k being the number of blocks and B being the size of the block. The goal would be maximize the distance between cluster and mninimazie distance within clusters. This could be done by word embedding like n-gram character embedding and nearest neighbour search Faiss.

3. To improve the system's performance, consider using more advanced NLP techniques like named entity recognition or part-of-speech tagging to identify company names in the input query if the input is a description of companies rather than a keyword. For dealing with ambiguity, Spacy entity linking can be a good method to deal with it.

4. Implementation and deployment: Consider integrating the system with a web or mobile application to make it accessible to users.